In [13]:
import pandas as pd
import re

# 데이터 불러오기
df_smishing = pd.read_csv('smishing.csv')
df_smishing = df_smishing[['Result']].copy()
df_smishing.rename(columns={'Result': 'text'}, inplace=True)
df_smishing['label'] = 1

# 정상 메시지 추출
ko_list = []
with open('normal.json', 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        match = re.search(r'"ko_original"\s*:\s*"([^"]+)"', line)
        if match:
            sentence = match.group(1)
            sentence = re.sub('^>+', '', sentence).strip()
            if len(sentence) >= 15:  # 글자 수 기준 설정
                ko_list.append(sentence)
        if len(ko_list) >= len(df_smishing):  # 2,842개만 수집
            break

# 정상 메시지 DataFrame
df_normal = pd.DataFrame({'text': ko_list})
df_normal['label'] = 0

# 4. 합치기
df = pd.concat([df_smishing, df_normal], ignore_index=True)

# 5. 셔플
df = df.sample(frac=1).reset_index(drop=True)

# 6. 저장
df.to_csv('merged_dataset.csv', index=False)
